## 4.4.3. Ewaluacja wyników klasyfikacji

#### Importy i ustawienia środowiska

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix

#### Wczytanie modelu i danych testowych

In [ ]:
model = tf.keras.models.load_model("resnet50_best_model.h5", compile=False)
df_test = pd.read_csv("./data/processed/images_to_train/resnet/test.csv")

#### Funkcja `load_image_test`
Funkcja odpowiedzialna za wczytanie obrazu, jego dekodowanie, skalowanie do rozmiaru 224×224 oraz normalizację do [0, 1].

In [ ]:
def load_image_test(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

#### Funkcja `make_test_dataset`
Funkcja odpowiedzialna za utworzenie zbioru, który wczytuje i przetwarza dane, grupuje w batch'e.

In [ ]:
def make_test_dataset(paths, labels, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image_test, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

#### Utworzenie zbioru danych gotowego do ewaluacji modelu.

In [ ]:
test_paths = df_test['path'].tolist()
test_labels = df_test['label_risk_group'].tolist()
test_ds = make_test_dataset(test_paths, test_labels)

#### Obliczenie predykcji, znalezienie najlepszego progu oraz wyliczenie metryk klasyfikacyjnych.

In [ ]:
y_true, y_pred_prob = [], []
for images, labels in test_ds:
    preds = model.predict(images).flatten()
    y_pred_prob.extend(preds)
    y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_pred_prob = np.array(y_pred_prob)

thresholds = np.linspace(0.1, 0.9, 81)
f1_scores = [f1_score(y_true, (y_pred_prob >= t).astype(int))
             for t in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred = (y_pred_prob >= best_threshold).astype(int)

f1 = f1_score(y_true, y_pred)
auc_pr_value = roc_auc_score(y_true, y_pred_prob)
auc_roc_value = tf.keras.metrics.AUC(curve='ROC')(y_true, y_pred_prob).numpy()
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"F1-score: {f1:.4f}")
print(f"AUC-PR: {auc_pr_value:.4f}")
print(f"AUC-ROC: {auc_roc_value:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Best threshold: {best_threshold:.2f}")

results = {
    "F1-score": f1,
    "AUC-PR": auc_pr_value,
    "AUC-ROC": auc_roc_value,
    "Sensitivity": sensitivity,
    "Specificity": specificity,
    "Best threshold": best_threshold
}

with open("classification_test_results.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Metric", "Value"])
    for key, value in results.items():
        writer.writerow([key, round(value, 4)])

#### Wizualizacja jakości modelu poprzez krzywą precyzja–czułość (Precision–Recall).

In [ ]:
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
auc_val = auc(recall, precision)

plt.figure()
plt.plot(recall, precision, label=f"AUC-PR = {auc_val:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve – Zbiór testowy")
plt.legend()
plt.grid(True)
plt.savefig("plots/pr_curve_test.png")